In [1]:
import keras

keras.__version__

'2.15.0'

In [2]:
from tensorflow.keras.utils import image_dataset_from_directory
from pathlib import Path
from config import DATASET
import tensorflow as tf

# Ensure dataset_path is a Path object
dataset_path = Path(DATASET)

train_dataset = image_dataset_from_directory(
    dataset_path / 'train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    seed=0,
    validation_split=0.2,
    subset='training'
)

validation_dataset = image_dataset_from_directory(
    dataset_path / 'train',
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    seed=0,
    validation_split=0.2,
    subset='validation'
)

# validation_split = 0.2
# total_batches = tf.data.experimental.cardinality(train_dataset).numpy()
# val_batches = int(validation_split * total_batches)

# train_dataset = train_dataset.skip(val_batches)
# validation_dataset = train_dataset.take(val_batches)


Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Found 100000 files belonging to 2 classes.
Using 20000 files for validation.


In [3]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [4]:
validation_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [5]:
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.data import AUTOTUNE

train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)

# caching/prefetching for better performance
# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])



94765736/94765736 [==============================] - 3s 0us/step


In [7]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [8]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)

Epoch 1/5


2500/2500 [==============================] - 2585s 1s/step - loss: 0.2166 - accuracy: 0.9112 - val_loss: 0.1403 - val_accuracy: 0.9442
Epoch 2/5
2500/2500 [==============================] - 6959s 3s/step - loss: 0.1534 - accuracy: 0.9406 - val_loss: 0.1192 - val_accuracy: 0.9534
Epoch 3/5
2500/2500 [==============================] - 5225s 2s/step - loss: 0.1369 - accuracy: 0.9462 - val_loss: 0.1094 - val_accuracy: 0.9577
Epoch 4/5
2500/2500 [==============================] - 3390s 1s/step - loss: 0.1217 - accuracy: 0.9528 - val_loss: 0.1051 - val_accuracy: 0.9600
Epoch 5/5
2500/2500 [==============================] - 5506s 2s/step - loss: 0.1125 - accuracy: 0.9573 - val_loss: 0.1055 - val_accuracy: 0.9603
